<a href="https://colab.research.google.com/github/alissonfariias/OCR-improvement/blob/main/T%C3%89CNICAS_PARA_APERFEI%C3%87OAMENTO_DE_RECONHECIMENTO_%C3%93TICO_DE_CARACTERES_EM_IMAGENS_DE_DOCUMENTOS_PESSOAIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📂 Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 📦 Bibliotecas necessárias

In [ ]:
!pip install pytesseract
!sudo apt-get install tesseract-ocr
!pip install opencv-python
!apt-get install tesseract-ocr-por
!pip install pillow
!pip install unidecode

## 📚 Importando as bibliotecas necessárias

In [ ]:
import cv2
import pytesseract
from PIL import Image
from google.colab import files
from IPython.display import Image
import os
import re
from unidecode import unidecode
import shutil
import zipfile
import difflib
import nltk
from google.colab import drive

## 📂 Organizando diretórios referentes aos documentos de RGs

In [ ]:
# Caminho para o diretório do TCC no Google Drive
tcc_directory = '/content/drive/My Drive/TCC/RG/'

# Caminho para o diretório de imagens
imagens_directory = os.path.join(tcc_directory, 'images_rg')

# Caminho para o diretório de ground_truth
ground_truth_directory = os.path.join(tcc_directory, 'ground_truth_rg')

# Diretório de saída para imagens pré-processadas
output_dir_preprocessed_images = '/content/preprocessed_images/'
os.makedirs(output_dir_preprocessed_images, exist_ok=True)

# Diretório de saída para arquivos OCR brutos
output_dir_ocr_files_raw = '/content/ocr_files_raw/'
os.makedirs(output_dir_ocr_files_raw, exist_ok=True)

# Diretório de saída para arquivos OCR com processamento
output_dir_ocr_files_processed = '/content/ocr_files_processed/'
os.makedirs(output_dir_ocr_files_processed, exist_ok=True)

# Listar arquivos de imagens no diretório
image_files = [f for f in os.listdir(imagens_directory) if f.endswith('.jpg')]

## 📂📄 Manipulação e pré-processamento dos arquivos 'ground truth' dos RGs

In [ ]:
# Criando diretório onde os arquivos ground truth originais serão modificados e armazenados
transcription_gt = '/content/transcription_gt'
os.makedirs(transcription_gt, exist_ok=True)

# Listar arquivos de ground_truth no diretório
ground_truth_files = [f for f in os.listdir(ground_truth_directory) if f.endswith('.txt')]

# Percorrendo os arquivos ground_truth carregados
for gt_filename in ground_truth_files:
    gt_path = os.path.join(ground_truth_directory, gt_filename)

    with open(gt_path, 'r', encoding='latin-1') as file:
        lines = file.readlines()

    # Removendo o nome da coluna (transcription)
    lines = lines[1:]

    # Extraindo a coluna "transcription"
    transcription_lines = [unidecode(line.strip().split(',')[-1].strip()) for line in lines]

    # Caminho para o novo arquivo
    new_gt_path = os.path.join(transcription_gt, gt_filename)

    # Escrever as linhas no novo arquivo
    with open(new_gt_path, 'w', encoding='latin-1') as new_file:
        new_file.writelines([line + '\n' for line in transcription_lines])

## 📄 Processamento e OCR de imagens dos RGs

In [ ]:
# Função para modificar o DPI da imagem e retorna a imagem com novo DPI.
def modify_dpi(image, dpi=500):
    new_width = int(image.shape[1] * (dpi / 300))
    new_height = int(image.shape[0] * (dpi / 300))

    # Redimensionar a imagem
    modified_image = cv2.resize(image, (new_width, new_height))

    return modified_image

In [ ]:
# Processar as imagens
for image_filename in image_files:
    image_path = os.path.join(imagens_directory, image_filename)
    image_name, image_extension = os.path.splitext(image_filename)

    image = cv2.imread(image_path)

    # Realizar OCR na imagem original
    textNO = pytesseract.image_to_string(image, lang='por')

    # Salvar os OCR's das imagens originais em um arquivo .txt
    output_text_raw_path = os.path.join(output_dir_ocr_files_raw, f"{image_name}.txt")
    with open(output_text_raw_path, "w") as text_file:
        text_file.write(textNO)

    # Modificar o DPI da imagem para 500
    modified_image = modify_dpi(image, dpi=500)

    # Aplicar suavização para remover ruído
    blurred_image = cv2.GaussianBlur(modified_image, (5, 5), 0)

    # Converter a imagem para escala de cinza
    gray_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2GRAY)

    # Salvar a imagem pré-processada
    output_image_path = os.path.join(output_dir_preprocessed_images, f"{image_name}.png")
    cv2.imwrite(output_image_path, gray_image)

    # Realizar OCR na imagem com pré-processamento
    text = pytesseract.image_to_string(gray_image, lang='por')

    # Removendo acentuação e deixando todas as letras MAÍUSCULAS
    text_preprocess = unidecode(text).upper()

    # Salvar os OCR's das imagens pré-processadas em um arquivo .txt
    output_text_path = os.path.join(output_dir_ocr_files_processed, f"{image_name}.txt")
    with open(output_text_path, "w") as text_file:
        text_file.write(text_preprocess)

## 📊 Avaliação OCRs (RGs)

In [ ]:
nltk.download('punkt')

# Caminhos para as pastas
pasta_pre_processamento = "/content/ocr_files_processed"
pasta_ocr_cru = "/content/ocr_files_raw"
pasta_ground_truth = "/content/transcription_gt"

# Função que calcula o F1 Score
def calcular_f1_score(precisao, revocacao):
    return 2 * (precisao * revocacao) / (precisao + revocacao) if (precisao + revocacao) > 0 else 0

# Listas para armazenar os valores F1-Score
f1_scores_pre = []
f1_scores_cru = []
nomes_arquivos_pre = []

for arquivo_ocr_pre, arquivo_ocr_cru, arquivo_gt in zip(os.listdir(pasta_pre_processamento), os.listdir(pasta_ocr_cru), os.listdir(pasta_ground_truth)):
    caminho_ocr_pre = os.path.join(pasta_pre_processamento, arquivo_ocr_pre)
    caminho_ocr_cru = os.path.join(pasta_ocr_cru, arquivo_ocr_cru)
    caminho_gt = os.path.join(pasta_ground_truth, arquivo_gt)

    with open(caminho_ocr_pre, 'r', encoding='utf-8') as file_ocr_pre, \
         open(caminho_ocr_cru, 'r', encoding='utf-8') as file_ocr_cru, \
         open(caminho_gt, 'r', encoding='utf-8') as file_gt:

        texto_ocr_pre = file_ocr_pre.read()
        texto_ocr_cru = file_ocr_cru.read()
        texto_gt = file_gt.read()

        # Calcular as métricas para o OCR com pré-processamento
        precisao_pre = difflib.SequenceMatcher(None, texto_ocr_pre, texto_gt).ratio()
        revocacao_pre = difflib.SequenceMatcher(None, texto_gt, texto_ocr_pre).ratio()
        f1_score_pre = calcular_f1_score(precisao_pre, revocacao_pre)
        f1_scores_pre.append(f1_score_pre)

        # Calcular as métricas para o OCR cru
        precisao_cru = difflib.SequenceMatcher(None, texto_ocr_cru, texto_gt).ratio()
        revocacao_cru = difflib.SequenceMatcher(None, texto_gt, texto_ocr_cru).ratio()
        f1_score_cru = calcular_f1_score(precisao_cru, revocacao_cru)
        f1_scores_cru.append(f1_score_cru)

# Calcular a média dos valores F1-Score
media_f1_pre = sum(f1_scores_pre) / len(f1_scores_pre)
media_f1_cru = sum(f1_scores_cru) / len(f1_scores_cru)

print(f'Média F1-Score OCR com pré-processamento e pós-processamento: {media_f1_pre:.2f}')
print(f'Média F1-Score OCR original: {media_f1_cru:.2f}')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Média F1-Score OCR com pré-processamento e pós-processamento: 0.53
Média F1-Score OCR original: 0.33


## 📂📄 Manipulação e pré-processamento dos arquivos 'ground truth' dos CPFs

In [ ]:
# Caminho para o diretório do TCC no Google Drive
tcc_directory = '/content/drive/My Drive/TCC/CPF/'

# Caminho para o diretório de ground_truth
ground_truth_directory = os.path.join(tcc_directory, 'ground_truth_cpf')

# Listar arquivos de ground_truth no diretório
ground_truth_files = os.listdir(ground_truth_directory)

# Expressão regular para encontrar CPFs
cpf_pattern = re.compile(r'\b\d{3}\.\d{3}\.\d{3}-\d{2}\b')

# Lista para armazenar todos os CPFs encontrados
all_cpfs = []

# Percorrendo os arquivos ground_truth carregados
for gt_filename in ground_truth_files:
    gt_path = os.path.join(ground_truth_directory, gt_filename)

    with open(gt_path, 'r', encoding='latin-1') as file:
        text = file.read()

    # Encontre todos os CPFs no texto usando a expressão regular
    cpfs = cpf_pattern.findall(text)

    # Adicione os CPFs à lista
    all_cpfs.extend(cpfs)

# Caminho para o novo arquivo único que conterá todos os CPFs no ambiente do Google Colab
output_file_path = '/content/all_cpfs_ground_truth.txt'

# Escrever todos os CPFs no novo arquivo, alternando entre as linhas
with open(output_file_path, 'w', encoding='latin-1') as output_file:
    for cpf in all_cpfs:
        output_file.write(cpf + '\n')

## 🆔 Processamento e OCR (com pré-processamento) das imagens dos CPFs


In [ ]:
# Função para modificar o DPI da imagem e retorna a imagem com novo DPI.
def modify_dpi(image, dpi=500):
    new_width = int(image.shape[1] * (dpi / 300))
    new_height = int(image.shape[0] * (dpi / 300))

    # Redimensionar a imagem
    modified_image = cv2.resize(image, (new_width, new_height))

    return modified_image

def extrair_numero_cpf(texto):
    cpf_regex = r'\d{3}\.\d{3}\.\d{3}-\d{2}'
    cpf_encontrado = re.search(cpf_regex, texto)

    if cpf_encontrado:
        return cpf_encontrado.group()
    else:
        return None

# Pasta que contém as imagens de CPFs
images_cpf = '/content/drive/MyDrive/TCC/CPF/images_cpf'

# Arquivo para salvar os números de CPF
output_file = 'ocr_all_cpfs_with_preprocess.txt'

# Abra o arquivo para escrever
with open(output_file, 'w') as file:
    for image_cpf_name in os.listdir(images_cpf):
        if image_cpf_name.endswith('.jpg'):
            caminho_imagem = os.path.join(images_cpf, image_cpf_name)
            imagem = cv2.imread(caminho_imagem)
            imagem_cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
            texto_reconhecido = pytesseract.image_to_string(imagem_cinza)
            numero_cpf = extrair_numero_cpf(texto_reconhecido)



            if numero_cpf is not None:
                file.write(numero_cpf + '\n')
            else:
                _, imagem_binaria = cv2.threshold(imagem, 60, 255, cv2.THRESH_BINARY)
                imagem_cinza = cv2.cvtColor(imagem_binaria, cv2.COLOR_BGR2GRAY)
                texto_reconhecido = pytesseract.image_to_string(imagem_cinza)
                numero_cpf = extrair_numero_cpf(texto_reconhecido)
                if numero_cpf is None:
                    imagem = cv2.imread(caminho_imagem)
                    modified_image = modify_dpi(imagem, dpi=700)
                    imagem_cinza = cv2.cvtColor(modified_image, cv2.COLOR_BGR2GRAY)
                    texto_reconhecido = pytesseract.image_to_string(imagem_cinza)
                    numero_cpf = extrair_numero_cpf(texto_reconhecido)
                    if numero_cpf is None:
                        numero_cpf = '000.000.000-00'
                        file.write(numero_cpf + '\n')
                    else:
                        file.write(numero_cpf + '\n')
                else:
                    file.write(numero_cpf + '\n')

## 📊 Avaliação OCRs (CPFs)

In [ ]:
# Caminho para o arquivo de CPFs gerado pelo ground_truth
ground_truth_file = '/content/all_cpfs_ground_truth.txt'

# Caminho para o arquivo de CPFs gerado pelo OCR
ocr_file = '/content/ocr_all_cpfs_with_preprocess.txt'

# Ler os CPFs do arquivo do ground_truth
with open(ground_truth_file, 'r', encoding='latin-1') as file:
    ground_truth_cpfs = set(file.read().splitlines())

# Ler os CPFs do arquivo do OCR
with open(ocr_file, 'r') as file:
    ocr_cpfs = set(file.read().splitlines())

# Calcular a interseção entre os CPFs do OCR e do ground_truth
intersection = ground_truth_cpfs.intersection(ocr_cpfs)

# Calcular a precisão (accuracy)
accuracy = len(intersection) / len(ground_truth_cpfs) * 100

print(f"Precisão (Accuracy): {accuracy:.2f}%")

# Calcular os CPFs que estão no ground_truth, mas não estão no OCR
missing_cpfs = ground_truth_cpfs.difference(ocr_cpfs)

# Calcular a taxa de erro (error rate)
error_rate = len(missing_cpfs) / len(ground_truth_cpfs) * 100

print(f"Taxa de Erro (Error Rate): {error_rate:.2f}%")

Precisão (Accuracy): 73.48%
Taxa de Erro (Error Rate): 26.52%
